## 1. Import Libraries

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from category_encoders import CountEncoder
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import os
import joblib

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

random_state = 4012

## 2. Load Data

In [10]:
X_train = pd.read_csv("../Processed Data Files/X_train.csv")
y_train = pd.read_csv("../Processed Data Files/y_train.csv")

## 3. Model

In [11]:
# Combine them!

scaler = StandardScaler()

onehot = OneHotEncoder(
    categories = [
            ['full time', 'contract', 'part time','flexi','other','unspecified'], # employment_type
            ['entry level', 'middle level', 'senior level', 'unspecified'], # required_experience
            ['high school or vocational degree', 'undergraduate', 'graduate', 'unspecified'], # required_education
    ],
    handle_unknown = 'ignore',  # <- Ignore unknown values (i.e. don't create a column for them)
)

freq_encoder = CountEncoder()

binary_columns = ['telecommuting', 'has_company_logo','has_questions', 'have_company_profile', 'have_requirements', 'have_benefits', 'have_category', 'high_salary']
numerical_columns = ['flesch_score_bin_ft','fk_grade_level_bin_ft', 'text_len']
onehot_columns = ['employment_type', 'required_experience','required_education']
freq_columns = ['location_country']

preprocessor = ColumnTransformer([
    # (nickname, transformer to apply, columns to apply to)
    ('binary', 'passthrough', binary_columns),  # <- 'passthrough' says to keep them but don't apply anything
    ('numerical', scaler, numerical_columns),   
    ('onehot', onehot, onehot_columns),
    ('frequency',freq_encoder, freq_columns),
],  remainder='passthrough')

In [12]:
pd.DataFrame(
    data = preprocessor.fit_transform(X_train),
    columns = preprocessor.get_feature_names_out(),
    index = X_train.index
)

,binary__telecommuting,binary__has_company_logo,binary__has_questions,binary__have_company_profile,binary__have_requirements,binary__have_benefits,binary__have_category,binary__high_salary,numerical__flesch_score_bin_ft,numerical__fk_grade_level_bin_ft,numerical__text_len,onehot__employment_type_full time,onehot__employment_type_contract,onehot__employment_type_part time,onehot__employment_type_flexi,onehot__employment_type_other,onehot__employment_type_unspecified,onehot__required_experience_entry level,onehot__required_experience_middle level,onehot__required_experience_senior level,onehot__required_experience_unspecified,onehot__required_education_high school or vocational degree,onehot__required_education_undergraduate,onehot__required_education_graduate,onehot__required_education_unspecified,frequency__location_country,remainder__0,remainder__1,remainder__2,remainder__3,remainder__4,remainder__5,remainder__6,remainder__7,remainder__8,remainder__9,remainder__10,remainder__11,remainder__12,remainder__13,remainder__14,remainder__15,remainder__16,remainder__17,remainder__18,remainder__19,remainder__20,remainder__21,remainder__22,remainder__23,remainder__24,remainder__25,remainder__26,remainder__27,remainder__28,remainder__29,remainder__30,remainder__31,remainder__32,remainder__33,remainder__34,remainder__35,remainder__36,remainder__37,remainder__38,remainder__39,remainder__40,remainder__41,remainder__42,remainder__43,remainder__44,remainder__45,remainder__46,remainder__47,remainder__48,remainder__49,remainder__50,remainder__51,remainder__52,remainder__53,remainder__54,remainder__55,remainder__56,remainder__57,remainder__58,remainder__59,remainder__60,remainder__61,remainder__62,remainder__63,remainder__64,remainder__65,remainder__66,remainder__67,remainder__68,remainder__69,remainder__70,remainder__71,remainder__72,remainder__73,remainder__74,remainder__75,remainder__76,remainder__77,remainder__78,remainder__79,remainder__80,remainder__81,remainder__82,remainder__83,remainder__84,remainder__85,remainder__86,remainder__87,remainder__88,remainder__89,remainder__90,remainder__91,remainder__92,remainder__93,remainder__94,remainder__95,remainder__96,remainder__97,remainder__98,remainder__99,remainder__100,remainder__101,remainder__102,remainder__103,remainder__104,remainder__105,remainder__106,remainder__107,remainder__108,remainder__109,remainder__110,remainder__111,remainder__112,remainder__113,remainder__114,remainder__115,remainder__116,remainder__117,remainder__118,remainder__119,remainder__120,remainder__121,remainder__122,remainder__123,remainder__124,remainder__125,remainder__126,remainder__127,remainder__128,remainder__129,remainder__130,remainder__131,remainder__132,remainder__133,remainder__134,remainder__135,remainder__136,remainder__137,remainder__138,remainder__139,remainder__140,remainder__141,remainder__142,remainder__143,remainder__144,remainder__145,remainder__146,remainder__147,remainder__148,remainder__149,remainder__150,remainder__151,remainder__152,remainder__153,remainder__154,remainder__155,remainder__156,remainder__157,remainder__158,remainder__159,remainder__160,remainder__161,remainder__162,remainder__163,remainder__164,remainder__165,remainder__166,remainder__167,remainder__168,remainder__169,remainder__170,remainder__171,remainder__172,remainder__173,remainder__174,remainder__175,remainder__176,remainder__177,remainder__178,remainder__179,remainder__180,remainder__181,remainder__182,remainder__183,remainder__184,remainder__185,remainder__186,remainder__187,remainder__188,remainder__189,remainder__190,remainder__191,remainder__192,remainder__193,remainder__194,remainder__195,remainder__196,remainder__197,remainder__198,remainder__199,remainder__200,remainder__201,remainder__202,remainder__203,remainder__204,remainder__205,remainder__206,remainder__207,remainder__208,remainder__209,remainder__210,remainder__211,remainder__212,remainder__213,remainder__214,remainder__215,remainder__216,remainder__217,rema

In [14]:
def train(x_train,y_train,model,params):
    
    gridsearchcv = GridSearchCV(model, params, cv=3, scoring='f1_weighted', verbose=True, n_jobs=-1)
    gridsearchcv.fit(x_train, y_train.values)
    best_model = model.set_params(**gridsearchcv.best_params_).fit(x_train, y_train.values)
    
    return best_model

In [15]:
model = Pipeline(steps=[
    ('preprocess', preprocessor),
    # ('resampling', SMOTEENN(random_state=random_state,enn=EditedNearestNeighbours(sampling_strategy='majority'))),
    ('train', DecisionTreeClassifier(random_state=random_state,class_weight='balanced')),
])

params = dict(
    train__criterion = ["gini","entropy","log_loss"],
    train__max_features = ['auto', 'sqrt', 'log2'],
    train__ccp_alpha = [0.1, .01, .001],
    train__max_depth = [5, 6, 7, 8, 9]
)

# train
# start = time.time()
best_model = train(X_train,y_train,model,params)
# end = time.time()

Fitting 3 folds for each of 135 candidates, totalling 405 fits


In [19]:
base_path = "../Models/" # path where the file is going to be saved in 

model_name = 'decision_tree'

model_path = os.path.join(base_path, f'{model_name}.joblib')

joblib.dump(best_model, model_path)

['../Models/decision_tree.joblib']